In [30]:
from statsbombpy import sb
import pandas as pd
import numpy as np

I denne notebook laves én stor graf for hvert hold i hver kamp.

In [ ]:
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

In [ ]:
#make a list to hold a df of the events of a match
df_lst = []

for id in events.match_id.unique():
    #find one match for one team
    match_subset = events.loc[events['match_id'] == id]
    match_subset = match_subset.loc[events['possession_team'] == 'Bayer Leverkusen']

    #include only passes and shots
    match_subset = match_subset.loc[(match_subset['type'] == 'Shot') | (match_subset['type'] =='Pass')]
    

    #de næste to linjer kan eventuelt kommenteres ud, og så kan vi bare fjerne na i stedet, hvis vi ikke vil have de ekstra nodes, jeg foreslår :D

    #if a pass does not have a recipient fill the field with the outcome. This allows us to create "out" or "incomplete" nodes if we wish
    match_subset.loc[match_subset['type'] =='Pass', 'pass_recipient_id'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_id']), x['pass_outcome'], x['pass_recipient_id']), axis=1)

    #add shot as a pass recipient as well
    match_subset.loc[match_subset['type'] =='Shot', 'pass_recipient_id'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_id']), x['shot_outcome'], x['pass_recipient_id']), axis=1)

    #sort the values like when we did the passing sequences
    match_subset = match_subset.sort_values(['period','timestamp'], ascending=[True, True])

    df_lst.append(match_subset)   

Så kan man mappe spillerne til deres positioner, og så bare lave alle graferne ved at iterere gennem listen, tænker jeg